In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time
import pickle
import re
# 드라이버
path = 'C:/Users/lenovo/Documents/aibootcamp/final_project/chromedriver.exe'
driver = webdriver.Chrome(service = Service(path))

In [2]:
# 원두 메인페이지
url = 'https://biz.koke.kr/products?filters=%7B%22categoryGroup%22%3A%22WholeBean%22%7D'
driver.get(url)
time.sleep(2)

In [ ]:
# 각각의 원두에 접근할 수 있는 url가져오기
beanUrlList = []

for page in range(31):
    item = driver.find_element(By.CSS_SELECTOR, 'div.grid.grid-cols-2.bg-koke-bg.border-t.md\:bg-transparent.md\:border-none.md\:gap-dist-lg.md\:grid-cols-3')    
    links = item.find_elements(By.CSS_SELECTOR, 'a.card-box')
    time.sleep(2)
    for i in range(len(links)):
        link = links[i].get_attribute('href')
        beanUrlList.append(link)
    
    if page != 31:
        next = driver.find_elements(By.CSS_SELECTOR ,"button.text-gray-500.bg-white.border.border-gray-200.koke-button")
        next[-1].click()
        time.sleep(5)

In [ ]:
len(beanUrlList)        # 원두 url list. 

In [ ]:
# Pickle로 beanUrlList을 저장
with open("links_559.pkl", "wb") as file:
    pickle.dump(beanUrlList, file)

In [3]:
# Load the set of beanUrlList from the file
with open("./links_559.pkl", "rb") as file:
    loaded_links = pickle.load(file)

In [4]:
beanUrlList = list(loaded_links)        # set타입을 list로 변경
beanUrlList

['https://biz.koke.kr/products/43',
 'https://biz.koke.kr/products/3028',
 'https://biz.koke.kr/products/4674',
 'https://biz.koke.kr/products/1316',
 'https://biz.koke.kr/products/4702',
 'https://biz.koke.kr/products/1341',
 'https://biz.koke.kr/products/1055',
 'https://biz.koke.kr/products/117',
 'https://biz.koke.kr/products/4692',
 'https://biz.koke.kr/products/3804',
 'https://biz.koke.kr/products/1343',
 'https://biz.koke.kr/products/3838',
 'https://biz.koke.kr/products/274',
 'https://biz.koke.kr/products/3030',
 'https://biz.koke.kr/products/3394',
 'https://biz.koke.kr/products/186',
 'https://biz.koke.kr/products/1706',
 'https://biz.koke.kr/products/4750',
 'https://biz.koke.kr/products/4306',
 'https://biz.koke.kr/products/1094',
 'https://biz.koke.kr/products/4312',
 'https://biz.koke.kr/products/1331',
 'https://biz.koke.kr/products/3513',
 'https://biz.koke.kr/products/3157',
 'https://biz.koke.kr/products/4485',
 'https://biz.koke.kr/products/3508',
 'https://biz.kok

In [34]:
# 실제 크롤링 해오는 부분
def coffeeInfo(link): 
    result = {}
    title = driver.find_element(By.CSS_SELECTOR, 'h1.text-3xl.tracking-tight.koke-word-keep-all').get_attribute('innerHTML')
    result['이름'] = title
    time.sleep(5)
    raw = driver.find_elements(By.CSS_SELECTOR,"dl.grid.bg-white.border-t.lg\:grid-cols-2")
    tab_dt = raw[0].find_elements(By.TAG_NAME,'dt')
    tab_dd = raw[0].find_elements(By.TAG_NAME, 'dd')
     
    dt_ls = []
    for i in range(len(tab_dt)):
        dt_ls.append(tab_dt[i].text)
        dt_ls.append(tab_dd[i].text)
    time.sleep(5)
    for i in range(int(len(dt_ls)/2)):
        result[dt_ls[2*i]] = dt_ls[ 2*i+1 ]
    time.sleep(5)            
    # product id
    pattern = r"https://biz.koke.kr/products/(\d+)"  # Regular expression pattern to match the product number
    match = re.search(pattern, link)  # link는 for i in ...에서 i 역할
    if match:
        product_number = match.group(1)
        result['ID'] = product_number  # 여기도 각자 코드에 맞춰서 넣으세요    
        
    print(result)
    return result
    # coffeeInfo_df.append(result, ignore_index=True)

In [35]:
results = []
for i in range(len(beanUrlList)):
    link = beanUrlList[i]
    time.sleep(10)
    driver.get(link)     # 해당 link 접속하기
    time.sleep(10)
    result = coffeeInfo(link)
    time.sleep(5)  
    results.append(result)

     

{'이름': '블루문', '식품의 유형': '커피', '제조원 및 소재지': '루비아 / 경기 성남시 사기막골로 124, 비즈동 1412호 루비아', '유통기한': '제조일로부터 1년', '제조일자': '주문 확인 후 제조', '내용량': '1kg', '보관방법': '직사광선을 피하고 온도, 습도가 낮으며 통풍이 잘 되는 곳에 밀폐 보관', '원재료 및 함량': '커피원두 100%', '제품문의 관련 주소 및 전화번호': '루비아 커피 / 010-2942-3069', 'ID': '3042'}
{'이름': '클래식 블렌드', '식품의 유형': '커피', '제조원 및 소재지': '엘카페커피로스터스, 서울시 영등포구 양평로21나길 7', '유통기한': '제조일로부터 1년', '제조일자': '제조일 별도표기', '내용량': '200g / 1kg', '보관 방법': '건냉한 곳에 밀폐보관', '원재료 및 함량': '원두커피 100%', '제품문의 관련 주소 및 전화번호': '코케 고객센터 070-4647-1868', 'ID': '140'}
{'이름': '밸런스', '식품의 유형': '커피', '제조원 및 소재지': '서울특별시 강남구 학동로 107, 5층', '유통기한': '제조일로부터 1년', '제조일자': '주문 확인 후 제조', '내용량': '500g/1kg', '보관방법': '직사광선을 피하고 온도, 습도가 낮으며 통풍이 잘 되는 곳에 밀폐 보관', '원재료 및 함량': '커피원두 100%', '제품문의 관련 주소 및 전화번호': '커피루소 02-2056-7619', 'ID': '3562'}
{'이름': '온두라스 COE#10 아구아 둘세 게이샤 워시드', '식품의 유형': '커피', '제조원 및 소재지': '경기도 부천시 소사구 호현로 489', '유통기한': '제조일로부터 1년', '제조일자': '주문 확인 후 제조', '내용량': '200g', '보관방법': '직사광선을 피하고 온도, 습도가 낮으며 통풍이 잘 되는 곳에 밀폐 보관', '원재료 및 함량': 

In [36]:
print(results)

[{'이름': '블루문', '식품의 유형': '커피', '제조원 및 소재지': '루비아 / 경기 성남시 사기막골로 124, 비즈동 1412호 루비아', '유통기한': '제조일로부터 1년', '제조일자': '주문 확인 후 제조', '내용량': '1kg', '보관방법': '직사광선을 피하고 온도, 습도가 낮으며 통풍이 잘 되는 곳에 밀폐 보관', '원재료 및 함량': '커피원두 100%', '제품문의 관련 주소 및 전화번호': '루비아 커피 / 010-2942-3069', 'ID': '3042'}, {'이름': '클래식 블렌드', '식품의 유형': '커피', '제조원 및 소재지': '엘카페커피로스터스, 서울시 영등포구 양평로21나길 7', '유통기한': '제조일로부터 1년', '제조일자': '제조일 별도표기', '내용량': '200g / 1kg', '보관 방법': '건냉한 곳에 밀폐보관', '원재료 및 함량': '원두커피 100%', '제품문의 관련 주소 및 전화번호': '코케 고객센터 070-4647-1868', 'ID': '140'}, {'이름': '밸런스', '식품의 유형': '커피', '제조원 및 소재지': '서울특별시 강남구 학동로 107, 5층', '유통기한': '제조일로부터 1년', '제조일자': '주문 확인 후 제조', '내용량': '500g/1kg', '보관방법': '직사광선을 피하고 온도, 습도가 낮으며 통풍이 잘 되는 곳에 밀폐 보관', '원재료 및 함량': '커피원두 100%', '제품문의 관련 주소 및 전화번호': '커피루소 02-2056-7619', 'ID': '3562'}, {'이름': '온두라스 COE#10 아구아 둘세 게이샤 워시드', '식품의 유형': '커피', '제조원 및 소재지': '경기도 부천시 소사구 호현로 489', '유통기한': '제조일로부터 1년', '제조일자': '주문 확인 후 제조', '내용량': '200g', '보관방법': '직사광선을 피하고 온도, 습도가 낮으며 통풍이 잘 되는 곳에 밀폐 보관', '원재료 및 함

In [40]:
# dictionary를 DataFrame 으로 만들기
df = pd.DataFrame(results)
print(df)

                      이름    식품의 유형                                 제조원 및 소재지  \
0                  데일리스윗  커피(볶은커피)              (주)핸디엄, 경기도 여주시 세종대왕면 상동길 68   
1                브라운렄블렌드        커피                     서울 강북구 한천로139가길 15 1층   
2             오렌지 슈가 블렌드        커피                   인천시 부평구 부평대로 337, 1031호   
3              블렌드 슈퍼 노멀        커피               서울특별시 서대문구 연희로 109, 2층 프로토콜   
4          과테말라 안티구아 SHB        커피                  충청남도 논산시 연산면 황룡재로 65, 2층   
..                   ...       ...                                       ...   
554         케냐 야라 AB TOP        커피                서울특별시 영등포구 문래로 20길 60 501호   
555  에티오피아 G2 시다모 SW 디카프     커피 원두                        파주시 꽃아마길41, 디카프워크룸   
556             브라우니 블렌드      원두커피         커넥츠커피, 서울특별시 마포구 성지길 60, 1층(국제빌딩)   
557                 룰라바이     원두 커피  영앤도터스, 서울 마포구 마포대로 156 공덕 프루지오시티 1층 107호   
558                  나탈리     원두 커피  영앤도터스, 서울 마포구 마포대로 156 공덕 프루지오시티 1층 107호   

                          유통기한         

In [41]:
# dataframe을 csv 파일로 저장하기
df.to_csv('./Required_information.csv', index = False, encoding='utf-8-sig')       # encoding='utf-8'이 한글 깨짐

In [20]:
# 크롤링 테스트 코드
raw = driver.find_elements(By.CSS_SELECTOR,"dl.grid.bg-white.border-t.lg\:grid-cols-2")
tab_dt = raw[0].find_elements(By.TAG_NAME,'dt')
tab_dd = raw[0].find_elements(By.TAG_NAME, 'dd')

dt_ls = []
for i in range(len(tab_dt)):
    dt_ls.append(tab_dt[i].text)
    dt_ls.append(tab_dd[i].text)
print(dt_ls)

['식품의 유형', '커피', '제조원 및 소재지', '경상북도 경산시 와촌면 갓바위로 67-2(와촌면) (우:38400)', '유통기한', '제조일로부터 1년', '제조일자', '주문일로부터 7일이내', '내용량', '1kg', '보관 방법', '직사광선을 피하고 밀봉하여 실온보관', '원재료 및 함량', '커피원두 100%(브라질 40%, 콜롬비아 40%, 에티오피아 20%)', '제품문의 관련 주소 및 전화번호', '코케 고객센터 070-4647-1868']


In [33]:
result = {}
for i in range(int(len(dt_ls)/2)):
    result[dt_ls[i]] = dt_ls[i + int(len(dt_ls)/2)]
result

{'식품의 유형': '커피',
 '제조원 및 소재지': '경기도 고양시 덕양구 행당로1 4층 401호',
 '유통기한': '제조일로부터 1년',
 '제조일자': '주문 확인 후 제조',
 '내용량': '1kg',
 '보관방법': '직사광선을 피하고 온도, 습도가 낮으며 통풍이 잘 되는 곳에 밀폐 보관',
 '원재료 및 함량': '커피원두 100%',
 '제품문의 관련 주소 및 전화번호': '컨트리사이드'}